In [1]:
import glob
import logging
import os

import catchment_delineation as cd
import numpy as np
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

# Terrain conditioning

For each vassdragsområde and DTM resolution, this notebook performs the following processing:

 * "Burn" [NVE lakes](https://kartkatalog.geonorge.no/metadata/innsjoedatabase/823b8639-9a49-41bf-8571-3608435eb149) and [ELVIS streams](https://kartkatalog.geonorge.no/metadata/elvis-elvenett/3f95a194-0968-4457-a500-912958de3d39) (optional)
 * Fill pits
 * Fill depressions
 * Resolve flats
 * Calculate flow direction
 * Calculate flow accumulation
 
The processing is done using [PySheds](https://github.com/mdbartos/pysheds).

## 1. User input

**Note:** The defaults in `pysheds.resolve_flats()` for `max_iter` and `eps` are 1000 and 1e-5, respectively. This caused strange results, where applying `resolve_flats` would fail to resolve all flats, and would also introduce additional pits, leading to very poor catchment delineation. The defaults seem to be appropriate only for fairly small areas (i.e. not that many grid cells). Setting `max_iter` to be very large and `eps` to be very small seems to resolve the problem, without too much additional overhead. See [here](https://github.com/mdbartos/pysheds/issues/188) for details.

**Note 2:** Some PySheds functions give strange output with irregularly DEMs containing NoData (e.g. non-rectangular datasets with NoData cells around the margins). Best results seem to be achieved by filling all NoData cells with valid low values. In particular, if no stream burning is being performed, set all NoData values and values less than zero to zero. Alternatively, if stream burning is included, set all NoData values and values less than zero to `-dz` (i.e. the stream burn depth). This essentially creates DEMs with large, low flat areas around the margins, which forces all cells to drain to them eventually.

In [2]:
res_list = [10]
no_data_val = -32767
dem_dtype = np.float32
burn = True
stream_sigma = None
stream_dz = 20
lake_dz = 20
max_iter = 1e9
eps = 1e-12
log_file = "terrain_conditioning_pysheds_10m.log"

n_jobs = 8

In [3]:
# Setup logging
logging.basicConfig(
    filename=log_file,
    format="%(asctime)s %(message)s",
    datefmt="%Y-%m-%d %H:%M",
    encoding="utf-8",
    filemode="w",
    level=logging.INFO,
)
print(f"Logging progress to '{log_file}'.")
logging.info("Started")

Logging progress to 'terrain_conditioning_pysheds_10m.log'.


## 2. Get vector data

### 2.1. ELVIS stream network

In [4]:
# %%time

# logging.info("Getting streams")
# # Convert generator to list so it can be reused in loop below
# stream_shapes = list(cd.get_elvis_streams_as_shapes(crs="epsg:25833"))

In [5]:
# import pickle

# with open(r"/home/jovyan/shared/01_datasets/spatial/streams.pkl", "wb") as f:
#     pickle.dump(stream_shapes, f)

### 2.2. Lake polygons

In [6]:
# %%time

# logging.info("Getting lakes")
# # Convert generator to list so it can be reused in loop below
# lake_shapes = list(cd.get_nve_lakes_as_shapes(crs="epsg:25833"))

In [7]:
# import pickle

# with open(r"/home/jovyan/shared/01_datasets/spatial/lakes.pkl", "wb") as f:
#     pickle.dump(lake_shapes, f)

### 2.3. Load saved data

In [8]:
%%time

import pickle

with open(r"/home/jovyan/shared/01_datasets/spatial/streams.pkl", "rb") as f:
    stream_shapes = pickle.load(f)

with open(r"/home/jovyan/shared/01_datasets/spatial/lakes.pkl", "rb") as f:
    lake_shapes = pickle.load(f)

CPU times: user 19.8 s, sys: 1.64 s, total: 21.5 s
Wall time: 21.5 s


## 3. Process vassdragsområder

### 3.1. Option 1: Process in series

In [9]:
# from importlib import reload
# cd = reload(cd)

for res in tqdm(res_list, desc="Looping over DTM resolutions"):
    logging.info(f"Processing {res} m DTM")
    search_path = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m/by_vassom/dtm/*.tif"
    flist = sorted(glob.glob(search_path))

    for fpath in tqdm(flist, desc="Looping over vassdragsområder"):
        fname = os.path.split(fpath)[1]
        vassom = fname.split("_")[1]

        logging.info(f"    Vassdragsområder {vassom}")

        fill_path = (
            f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m"
            f"/by_vassom/dtm_fill_burn/vassom_{vassom}_{res}m_burn_fill.tif"
        )
        fdir_path = (
            f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m"
            f"/by_vassom/flow_direction/vassom_{vassom}_{res}m_fdir.tif"
        )
        facc_path = (
            f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m"
            f"/by_vassom/flow_accumulation/vassom_{vassom}_{res}m_facc.tif"
        )

        cd.condition_dem(
            fpath,
            fill_path,
            fdir_path,
            facc_path,
            dem_dtype=dem_dtype,
            dem_ndv=no_data_val,
            burn=burn,
            stream_shapes=stream_shapes,
            lake_shapes=lake_shapes,
            stream_sigma=stream_sigma,
            stream_dz=stream_dz,
            lake_dz=lake_dz,
            max_iter=max_iter,
            eps=eps,
        )
logging.info("Done.")

Looping over DTM resolutions:   0%|          | 0/1 [00:00<?, ?it/s]

Looping over vassdragsområder:   0%|          | 0/262 [00:00<?, ?it/s]

        vassom_002_10m_burn_fill.tif has 105 pits and 161 flats.
        vassom_005_10m_burn_fill.tif has 3 pits and 0 flats.
        vassom_008_10m_burn_fill.tif has 3 pits and 0 flats.
        vassom_009_10m_burn_fill.tif has 3 pits and 0 flats.
        vassom_014_10m_burn_fill.tif has 6 pits and 0 flats.
        vassom_015_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_022_10m_burn_fill.tif has 2 pits and 0 flats.
        vassom_024_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_028_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_029_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_030_10m_burn_fill.tif has 4 pits and 0 flats.
        vassom_034_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_035_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_039_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_042_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_043_10m_burn_fill.tif has 1 pits and 0 flats.
        vassom_045_1

### 3.2. Option 2: Process in parallel

(But note that logging doesn't work properly).

In [10]:
# %%time


# for res in tqdm(res_list, desc="Looping over DTM resolutions"):
#     logging.info(f"Processing {res} m DTM")
#     search_path = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m/by_vassom/dtm/*.tif"
#     flist = sorted(glob.glob(search_path))
#     vassom_list = [os.path.split(fname)[1].split("_")[1] for fname in flist]

#     fill_fold = (
#         f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m"
#         f"/by_vassom/dtm_fill_burn"
#     )
#     fill_paths = [
#         os.path.join(fill_fold, f"vassom_{vassom}_{res}m_burn_fill.tif")
#         for vassom in vassom_list
#     ]

#     fdir_fold = (
#         f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m"
#         f"/by_vassom/flow_direction"
#     )
#     fdir_paths = [
#         os.path.join(fdir_fold, f"vassom_{vassom}_{res}m_fdir.tif")
#         for vassom in vassom_list
#     ]

#     facc_fold = (
#         f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m"
#         f"/by_vassom/flow_accumulation"
#     )
#     facc_paths = [
#         os.path.join(facc_fold, f"vassom_{vassom}_{res}m_facc.tif")
#         for vassom in vassom_list
#     ]

#     Parallel(n_jobs=n_jobs)(
#         delayed(cd.condition_dem)(
#             src_path,
#             fill_paths[idx],
#             fdir_paths[idx],
#             facc_paths[idx],
#             dem_dtype=dem_dtype,
#             dem_ndv=no_data_val,
#             burn_streams=True,
#             shapes=shapes,
#             sigma=sigma,
#             dz=dz,
#             max_iter=max_iter,
#             eps=eps,
#         )
#         for idx, src_path in enumerate(flist)
#     )